In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import pandas as pd
import albumentations as A
from albumentations import *
from albumentations.pytorch.transforms import ToTensorV2
import effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, create_model_from_config
from effdet.efficientdet import HeadNet

from datetime import datetime
import time
import random
import cv2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import GradScaler, autocast
from glob import glob

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(0)

In [5]:
labelmap = {'fruit_healthy':1, 'fruit_woodiness':2, 'fruit_brownspot':3}
labelmap_inv = {k:v for v,k in labelmap.items()}

class PassionFruitDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, img_path, transforms):
        self.csv = pd.read_csv(csv_path)
        self.root = img_path
        self.transforms = transforms

    def __getitem__(self, idx):
        # load images and masks
        row = self.csv.iloc[idx]
        img_path = os.path.join(self.root, row.Image_ID) + '.jpg'
        img = np.array(Image.open(img_path).convert("RGB")).astype('uint8')
        #img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
        if self.transforms is not None:
            annotations = {'image': img}
            augmented = self.transforms(**annotations)
            img = augmented['image']
        
        return img/255, row.Image_ID

    def __len__(self):
        return len(self.csv)
    
def vis(img, target):
    visualize.show_labeled_image(img.permute(1,2,0).numpy(), target['boxes'].numpy(), target["labels"])

In [6]:
transform_test = Compose([
    #Normalize(),
    ToTensorV2(),
])

val_ds = PassionFruitDataset(
    'data/Test.csv',
    'data/Test_Images/Test_Images', 
    transform_test
)

data_loader = DataLoader(
    val_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=False
)

In [7]:
def get_net(model_name):
    if model_name == 'resnet':
        backbone = torch.nn.Sequential(*(list(torchvision.models.resnet101(pretrained=True).children())[:-2]))
        backbone.out_channels = 2048
    elif model_name == 'densenet':
        backbone = torchvision.models.densenet121(pretrained=True).features
        backbone.out_channels = 1024
    
    elif model_name == 'inception':
            backbone = torch.nn.Sequential(*(list(torchvision.models.inception_v3(pretrained=True).children())[:-3]))
            backbone.out_channels = 1024
    
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0','1','2','3'],
                                                    output_size=7,
                                                    sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                       num_classes=4,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)
    return model

In [8]:
def make_predictions(images, score_threshold=0.2):
    images = images.cuda().float()
    predictions = []
    net.cuda()
    net.eval()
    with torch.no_grad():
        det = net(images)
        for i in range(images.shape[0]):
            boxes = det[i]['boxes'].detach().cpu().numpy()   
            scores = det[i]['scores'].detach().cpu().numpy()
            classes = det[i]['labels'].detach().cpu().numpy()
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            
           
            predictions.append({ 
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'classes': classes[indexes]
            })
    return [predictions]

def iou(box):
    mat = []
    ious = []
    for a in range(len(box)):
        for b in range(len(box)):
            if a != b:
                boxA = box[a]
                boxB = box[b]
                xA = max(boxA[0], boxB[0])
                yA = max(boxA[1], boxB[1])
                xB = min(boxA[2], boxB[2])
                yB = min(boxA[3], boxB[3])
                interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
                boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
                boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
                iou = interArea / float(boxAArea + boxBArea - interArea)
                if iou not in ious:
                    ious.append(iou)
                    mat.append([a,b])
    return np.array(ious), np.array(mat)

def postprocessing(boxes, scores, labels):
    boxes = (boxes).astype(np.int32).clip(min=0, max=512)
    ious, mat = iou(boxes)
    del_index = []
    for i in mat[ious > 0.8]:
        if scores[i[0]] > scores[i[1]]:
            del_index.append(i[1])
        else:
            del_index.append(i[0])
    boxes = np.delete(boxes,del_index,axis = 0)
    scores = np.delete(scores,del_index)
    labels = np.delete(labels,del_index)
    
    return boxes, scores, labels

In [17]:
net = get_net('resnet')
checkpoint = torch.load('logs/fasterrcnn-resnet101-pseudo-heavyaug-5add32_64-5ep-final/best-checkpoint-001epoch.bin')
net.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [18]:
results = []

for images, image_id in data_loader:
    predictions = make_predictions(images,0)
    
    for i, image in enumerate(images):
        boxes, scores, labels = predictions[0][i]['boxes'], predictions[0][i]['scores'], predictions[0][i]['classes']
        
        boxes, scores, labels = postprocessing(boxes, scores, labels)
        result = {
            'image_id': image_id[i],
            'boxes': boxes,
            'labels': labels,
            'scores': scores,
        }
        results.append(result)

In [21]:
train = pd.read_csv('data/labels_train.csv')
train_max = max(train.image_id)

In [22]:
sub = pd.DataFrame(columns = ['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax'])
tr = 0.0
img_name = ''
counter2 = 0
for i in results:
    ID = i['image_id']
    boxes = i['boxes']
    scores = i['scores']
    
        
    mask = scores > tr

    if not mask.any():
        mask[np.argmax(scores)] = True
    labels = i['labels'][mask].tolist()
    boxes = i['boxes'][mask]
    labels = [labelmap_inv[j] for j in i['labels']]

    for j, box in enumerate(boxes):

        sub.loc[counter2] = [ID,
                             labels[j],
                             scores[j],
                             box[1],
                             box[0],
                             box[3],
                             box[2]]
        counter2 += 1

In [25]:
sub.to_csv('concat/resnet101.csv',index = False)